In [ ]:
# Change the category (short-hand) names here if you prefer something else

categories = ["Data Quality", "Data Access", "UX", "Regulatory", "Practicality", "Track Record"]

In [ ]:
from pathlib import Path

import pandas as pd
import altair as alt
import numpy as np

import ideafast_deviceselection as ifds

source = Path(ifds.__file__).parent.parent.absolute() / 'local/DeviceSelectionScoring.xlsx'


df = pd.read_excel(
    f"{source}",
    index_col=0, header=[0,1], nrows=69,
    sheet_name='MASTER SCORES')
df.drop(df.columns[2:12],axis=1,inplace=True)
df.drop(df.columns[0:1],axis=1,inplace=True)
df.drop(df.index[0],axis=0,inplace=True)
df.drop(df.columns[31:],axis=1,inplace=True)

# extend empty headers to use the multiIndex
a = df.columns.get_level_values(0).to_series().mask(lambda x: x.str.startswith('Unnamed')).ffill()
a[0] = 'relevance'
b = df.columns.get_level_values(1)
df.columns = [a, b]

df.index.names = [('criteria','type')]
df.columns.names = ['device','type']

df.drop('RELEVANCE * SCORE',axis=1,level=1,inplace=True)

df.reset_index()

In [ ]:
df_stacked = df.set_index([('relevance','relevance score')], append=True)

df_stacked= df_stacked.stack(level=0, dropna=False).rename_axis(index={('criteria','type'): 'criteria',('relevance','relevance score'):'relevance'}).reset_index()
df_stacked.insert(1,'group',[categories[int(x)-1] for x in df_stacked['criteria'].str[0]])

df_stacked

In [ ]:
# non-weighted scores are already in the dataframe
# scored = long.loc[(long['type'] == "SCORE") | (long['type'] == "CERTAINTY")]

# for weighted scores in a boxplot, we want to express the weight in 'observations'
# i.e., a relevance of 5 is represented by 5 rows with the same score.
nested_result = [int(x.relevance) * (x, ) for x in df_stacked.itertuples(index=False) if not np.isnan(x.SCORE) and not np.isnan(x.CERTAINTY)]

# !! Howcome there is a differente in x.SCORE is nan's and x.CERTAINTY is nan's??

result = [element for tupl in nested_result for element in tupl]
scored_weighted = pd.DataFrame(result)

scored_weighted

In [ ]:
from altair import datum


# Filter to get specific devices only here:
# of_interest = ['VTT EBED','ZKOne','Dreem']
#of_interest = ['McRoberts','Axivity']
#of_interest = ['Vital Patch']
of_interest = ['Cambridge Cognition', 'VTT SMA']
# of_interest = ['Vital Patch']
of_interest = ['VTT EBED','ZKOne','Dreem']
of_interest = ['Biovotion Everion']
of_interest = ['Vital Patch','VTT EBED','VTT SMA','Cambridge Cognition','Axivity','McRoberts','ZKOne','Biovotion Everion','Dreem','Byteflies']


scored_weighted_filtered = scored_weighted[scored_weighted['device'].isin(of_interest)]
df_stacked_filtered = df_stacked[df_stacked['device'].isin(of_interest)]


base = alt.Chart(scored_weighted_filtered)
base_no_relevance_weighted = alt.Chart(df_stacked_filtered)

boxplot = base.mark_boxplot(
    # comment-out the extent to hide outliers
    # default outliers are 1.5 x IQR (Inter Quartile Range)
    # extent='min-max',
    median=False
).encode(
    x=alt.X('group:O', axis=alt.Axis(labels=False, title=None), scale=alt.Scale(domain=sorted(categories))),
    y= alt.Y('SCORE:Q',
        title='Score'
    ),
    tooltip=alt.Tooltip("mean(SCORE):Q", format=",.2f",),
    # comment-out color to remove certainty representation
    color=alt.Color('mean(CERTAINTY)',
                    scale=alt.Scale(scheme='lighttealblue'),
                    title='Certainty'
                   ),
).properties(
    height=250
)

mean_tick = boxplot.mark_tick(thickness=2, width=15).encode(
    y='mean(SCORE)', 
    text='mean(SCORE)',
    color=alt.value('coral')
)

bars = base_no_relevance_weighted.mark_bar(
    color='coral'
).transform_calculate(
  baseline='0'
).encode(
    x=alt.X('group:O', axis=alt.Axis(title=None), scale=alt.Scale(domain=sorted(categories))), 
    y= alt.Y('valid(SCORE)',
        scale=alt.Scale(domain=(5, 0)),
        title='Flags',
        axis=alt.Axis(tickCount=5.0)
    ),
    y2='baseline:Q',
).properties(
    height=50
).facet(
    alt.Column("device:O", title=None, header = alt.Header(labelExpr="''"))  # explain the 'weighted' part
).transform_filter(
   (datum.relevance > 4) &  (datum.SCORE < .5)
)


final_boxplot = (boxplot + mean_tick).facet(
    alt.Column("device:O") #, title="Criteria score distribution with relevance weighted as repeated observations")  # explain the 'weighted' part
)

final_chart = (final_boxplot & bars).configure_concat(
    spacing=-10
)

final_chart

In [ ]:
# NOTE: this is using weighted score, but not certainty weighted

of_interest = ['Axivity', 'McRoberts']
of_interest = ['Cambridge Cognition', 'VTT SMA']
of_interest = ['Vital Patch']
of_interest = ['VTT EBED','ZKOne','Dreem']
of_interest = ['Biovotion Everion']
of_interest = ['Vital Patch','VTT EBED','VTT SMA','Cambridge Cognition','Axivity','McRoberts','ZKOne','Biovotion Everion','Dreem','Byteflies']

scored_weighted_filtered = scored_weighted[scored_weighted['device'].isin(of_interest)]

heatmap = alt.Chart(scored_weighted_filtered).mark_rect().encode(
    x='group:O',
    y='device:O',
    color=alt.Color('mean(SCORE)',
                    scale=alt.Scale(
                        # change the color scheme if preferred, choose from https://vega.github.io/vega/docs/schemes/
                        scheme='redyellowgreen', 
                        # change the domain to influence what is the 'midpoint'
                        domain=['0.2','1.0']
                    ),
                    title='Score'
                   ),
)

text = alt.Chart(scored_weighted_filtered).mark_text(color='black').encode(
    x=alt.X('group:O', title='Technology Selection Categories (TSC)'),
    y=alt.Y('device:O', title='Technologies'),
    text=alt.Text('mean(SCORE)', format=',.2f'),
)

(heatmap + text).properties(
    width=300,
    height=150
)